In [1]:
import sys
import os
from os.path import join
from tqdm.notebook import tqdm
import numba

import numpy as np
import awkward as ak

sys.path.insert(0, '../MATHUSLA_FastSim/')
import DetectorSimulation.Detector as Detector
import DetectorSimulation.Particle as Particle
import DetectorSimulation.Vertex as Vertex
import DetectorSimulation.lorentz_transformation as lt
import DetectorSimulation.llp_gun as llp_gun
## depending on which fvs are used load a different specialized llp gun (llp_gun_new) (built on top of llp_gun)
import DetectorSimulation.llp_gun_new as lg
from Helpers.functions import *

sys.path.insert(0,'../FastSim_Additions/')
from Additions import initiate_detector

In [2]:
fv_path = join(os.getcwd(), "../SimulationData/RHN_Utau_LLPweight4vectorBmesonlist_mN_0.1.csv")
vectors = read_vectors(fv_path, 50)

In [3]:
detector_benchmark = initiate_detector()
phi_min, phi_max, theta_min, theta_max = get_detector_angles(detector_benchmark)

In [4]:
def get_decay_products():
    products = [[13,13], [11,11], [11,13], [11,12,13], [12,12,13]]
    index = np.random.randint(0, len(products))
    return products[index]

In [5]:
np.array(vectors)[:,1:]

array([[-5.89417714e-01, -8.44930968e-02,  1.64605859e-01,
         5.50616990e-01],
       [-6.01994088e-01, -7.36149759e-02,  5.12447304e-01,
        -2.90474573e-01],
       [-2.09029952e+00,  1.87396231e-01,  1.98088328e+00,
        -6.32721232e-01],
       [-1.95372003e+00,  1.63880437e+00, -6.31572425e-01,
        -8.49975569e-01],
       [-2.89373251e+00,  3.93116560e-01, -1.82437338e+00,
         2.20925530e+00],
       [-2.77387863e+01, -1.93927025e-01,  3.34907028e+00,
        -2.75350029e+01],
       [-9.13000630e-01,  2.75152235e-01, -1.08765942e-01,
         8.57922705e-01],
       [-5.93354682e-01,  3.47657893e-01, -2.39729342e-02,
        -4.69711685e-01],
       [-1.01197275e+00,  4.06140908e-01,  5.44352710e-01,
        -7.43517678e-01],
       [-9.75722192e+01,  7.29014423e-01,  1.31622341e+01,
         9.66775677e+01],
       [-1.45502647e+00, -8.76318885e-01,  7.67905821e-01,
         8.65729680e-01],
       [-8.78067178e+00,  1.94709675e-01, -7.85562961e-01,
      

In [6]:
ctau = 100
mass = 1

storage = []

for k in tqdm(range(len(vectors))):
    
    llp_4p = vectors[k][1:]
    llp_theta = get_theta(llp_4p[1:])
    
    if (llp_theta < theta_max) and (llp_theta > theta_min):
        detector_benchmark.clear_detector()
        
        rotated_llp_4p = deal_with_phi(llp_4p, phi_min, phi_max)
        pack = get_weight(rotated_llp_4p, mass, ctau, detector_benchmark)
        
        if pack is not None:
            p_decay, p_pos, boost = pack
            
            decay_products = get_decay_products()
            if len(decay_products) == 2:
                llp_vertex = lg.get_llp('leptonic2body', mass, p_pos, boost, decay_products)
            else:
                llp_vertex = lg.get_llp('leptonic3body', mass, p_pos, boost, decay_products)
                
            if llp_vertex is not None:
                detector_benchmark.new_vertex_event(llp_vertex)

                storage.append(detector_benchmark.return_current_vertex())
                

  0%|          | 0/50 [00:00<?, ?it/s]

In [7]:
for_study = detector_benchmark.return_current_vertex()

In [8]:
for_study.vertex.momentum

array([1.92931216, 0.27339883, 1.52997254, 0.55378924])

In [9]:
for_study.particles[1].tracker_hits

[]

In [10]:
daughters = []

for daughter in for_study.particles:
    print(daughter.particle.pid)
    print(daughter.particle.momentum)
    print(len(daughter.tracker_hits))
    info = ak.Array([daughter.particle.pid,
              daughter.particle.momentum,
              len(daughter.tracker_hits)])
    daughters.append(info)

11
[0.41187378 0.1737054  0.29522351 0.22871218]
1
12
[ 0.1467769   0.10901865 -0.0756024  -0.06279068]
0
13
[ 1.37066148 -0.00932522  1.31035143  0.38786775]
1


In [11]:
ak.Array(daughters)

<Array [[11, [0.412, ..., 0.229], 1], ...] type='3 * var * union[int64, var...'>

In [12]:
for_study.recon_criteria

[]

In [12]:
proposed_struct = ak.Array([for_study.vertex.position,
                            for_study.vertex.momentum,
                            daughters,
                            for_study.recon_criteria,
                            for_study.wall_hit])

In [13]:
sys.path.insert(0, '../FastSim_Additions')
from Event_Data_Structure import *

In [14]:
storage

In [15]:
test = multi_event_data(storage)

In [9]:
data_to_pickle(test, 'FirstPkl.pickle')

In [10]:
with open('FirstPkl.pickle', 'rb') as f:
    data_in = pickle.load(f)

In [52]:
test[:,1]

<Array [[1.16, 0.00573, 0.512, 0.3], ...] type='10 * union[var * union[floa...'>

In [57]:
llppos = ak.Array([[1.59, 142, 83.1],
 [-21.3, 78.4, 60.3]])
llpp4 = ak.Array([[1.16, 0.00573, 0.512, 0.3],
 [1.41, -0.209, 0.772, 0.594]])
ak.cartesian([llppos,llpp4])
alldata = ak.zip({'llppos':llppos,'llpp4':llpp4})

ValueError: cannot broadcast nested list

In [54]:
alldata["llppos"]

<Array [[1.59, 142, 83.1], [-21.3, ..., 60.3]] type='2 * var * float64'>

In [14]:
print(data_in['Header'])

File contains vertex events from a simulation run 
The first key in data will be the event number. 
Each event has keys:
0 -> Position, 1 -> LLP Momentum, 2 -> Daughter Particles, 3 -> Vertex Reconstruction, 4 -> Wall Hit Information


In [15]:
data_in['Data']

<Array [[[48.4, 139, 61], ..., {...}], ...] type='10 * var * union[var * un...'>